In [1]:
#Se crea la conexion a la base de datos utilizando el patron Singleton para garantizar una unica conexion y se realizan todas las importaciones necesarias

import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'Avance2/tests')))

from patterns.singleton_SQL import MySQLConnection
from patterns.builder_SQL import QueryBuilder
from patterns.strategy_SQL import KPIContext, TotalSalesKPI, SalesByCategoryKPI, CustomersCountKPI
from test_patterns import run_all_tests

conn1 = MySQLConnection()


In [13]:
#CTE para obtener el total de ventas por producto, y luego filtrar los productos que hayan tenido más de 100 ventas.

query = """
        WITH ventas_por_producto AS (
            SELECT 
                ProductID,
                COUNT(*) AS total_ventas
            FROM sales
            GROUP BY ProductID
        )
        SELECT 
            vp.ProductID,
            vp.total_ventas,
            p.ProductName
        FROM ventas_por_producto vp
        JOIN products p ON p.ProductID = vp.ProductID
        WHERE vp.total_ventas > 100;
        """
df = conn1.execute_query(query)
print(df)

     ProductID  total_ventas                      ProductName
0            1           108              Flour - Whole Wheat
1            2           114       Cookie Chocolate Chip With
2            3           111               Onions - Cippolini
3            4           123       Sauce - Gravy; Au Jus; Mix
4            5           101           Artichokes - Jerusalem
..         ...           ...                              ...
371        446           101               Sunflower Seed Raw
372        447           102       Salmon - Atlantic; Skin On
373        450           132    Wine - Vidal Icewine Magnotta
374        451           125  Soup - Campbells Tomato Ravioli
375        452           122          Napkin White - Starched

[376 rows x 3 columns]


In [18]:
#Funcion ventana para cada venta, mostrar el total acumulado de ventas por cliente (customer_id) ordenado cronológicamente.

query = """
        SELECT
        CustomerId,
        SalesDate,
        Quantity,
        SUM(Quantity) OVER (PARTITION BY CustomerId ORDER BY SalesDate) AS acumulado_cliente
        FROM sales; 
        """
df = conn1.execute_query(query)
print(df)

       CustomerId       SalesDate  Quantity acumulado_cliente
0               1 0 days 23:37:01         1                 1
1               1 2 days 07:10:00         1                 2
2               4 1 days 04:56:00         1                 1
3               8 0 days 11:43:00         1                 1
4              10 0 days 15:16:00         1                 1
...           ...             ...       ...               ...
49995       98749 1 days 09:20:00        25                25
49996       98750 1 days 14:20:01        25                25
49997       98751 1 days 06:33:00        25                25
49998       98752 1 days 19:02:00        25                25
49999       98756 2 days 09:40:00        25                25

[50000 rows x 4 columns]


In [ ]:
#Se utiliza el patron Builder para ir armando dinamicamente una consulta
builder = QueryBuilder()
query = (builder.select("c.CategoryName", "COUNT(s.TotalPrice) AS total_sales")
               .from_table("sales s")
               .join("JOIN products p ON s.ProductID = p.ProductID")
               .join("JOIN categories c ON p.CategoryId = c.CategoryId")
               .group_by("c.CategoryName")
               .order_by("total_sales DESC")
               .limit(3)
               .build())
print("Consulta construida con Builder:")
print(query)

df_builder = conn1.execute_query(query)
print("\nResultado de la consulta Builder:")
print(df_builder)

Consulta construida con Builder:
SELECT c.CategoryName, COUNT(s.TotalPrice) AS total_sales FROM sales s JOIN products p ON s.ProductID = p.ProductID JOIN categories c ON p.CategoryId = c.CategoryId GROUP BY c.CategoryName ORDER BY total_sales DESC LIMIT 3

Resultado de la consulta Builder:
  CategoryName  total_sales
0  Confections         6299
1         Meat         5486
2      Poultry         5156


In [ ]:
#Se utiliza el patron Strategy para armar distintos KPIs

kpi_context = KPIContext(TotalSalesKPI(conn1))
total_sales = kpi_context.get_kpi()
print(f"Total ventas: ${total_sales}")

kpi_context = KPIContext(CustomersCountKPI(conn1))
total_customers = kpi_context.get_kpi()
print(f"Total customers: {total_customers}")

kpi_context = KPIContext(SalesByCategoryKPI(conn1))
sales_category = kpi_context.get_kpi()
print("Sales by category:")
print(f"{sales_category}")

Total ventas: $7863619.00
Total customers: 98759
Sales by category:
   CategoryName  total_sales
0   Confections         6299
1          Meat         5486
2       Poultry         5156
3       Cereals         5110
4       Produce         4771
5     Beverages         4196
6        Snails         4119
7       Seafood         4059
8    Shell fish         3949
9         Dairy         3892
10        Grain         2963


In [2]:
#Se corre esta funcion general de test que lo que hace es correr todos los tests para el avance 2 y se termina la conexion a la base
run_all_tests()
conn1.close_engine()


✔ test_singleton_connection passed
✔ test_builder_query_structure passed
✔ test_strategy_total_sales_kpi passed
✔ test_strategy_sales_by_category_kpi passed
✔ test_strategy_customers_count_kpi passed
✅ Todas las pruebas pasaron correctamente.
Finalizada la conexion a la base
